In [1]:
import datetime
from pathlib import Path

from pydantic_ai import Agent
from pydantic_ai import messages as _messages

from knd.ai import replace_system_parts, trim_messages
from knd.memory import Memories, Memory

In [2]:
aot1 = Memory(
    context="specifically the favorite character in attack on titan",
    content="Eren Yeager",
    category="preference",
)
aot2 = Memory(
    context="favorite character in attack on titan",
    content="Mikasa Ackerman",
    category="preference",
    superseded_ids=[str(aot1.id)],
)
mems = Memories(memories=[aot1, aot2])

In [3]:
mems.model_dump()

{'memories': [{'id': UUID('ae53ffd7-0cbc-47d4-8bfb-fa96cdbceab8'),
   'created_at': datetime.datetime(2025, 1, 13, 23, 37, 46, 800309),
   'context': 'favorite character in attack on titan',
   'category': 'preference',
   'content': 'Mikasa Ackerman',
   'superseded_ids': []}]}

In [4]:
a = [1, 2, 3]
a[-3:]


[1, 2, 3]

In [13]:
Memory(content="hello", context="context")

Memory(id=UUID('04cb2fc4-d5e8-4c63-938d-66900886148d'), created_at=datetime.datetime(2025, 1, 13, 23, 22, 27, 742164), context='context', content='hello')

In [9]:
mems.model_dump()

{'memories': [{'id': UUID('04cb2fc4-d5e8-4c63-938d-66900886148d'),
   'created_at': datetime.datetime(2025, 1, 13, 23, 0, 19, 717946),
   'context': 'context',
   'content': 'hello'}]}

In [11]:
mems = Memories.model_validate_json(Path("mems.json").read_text())
mems.model_dump()


{'memories': [{'id': UUID('04cb2fc4-d5e8-4c63-938d-66900886148d'),
   'created_at': datetime.datetime(2025, 1, 13, 23, 0, 19, 717946),
   'context': 'context',
   'content': 'hello'}]}

In [3]:
messages = [
    _messages.ModelRequest(
        parts=[
            _messages.SystemPromptPart(
                content="<ROLE>\nYou are an experienced investment advisor specializing in creating detailed investor profiles. Your primary responsibility is to engage with users in a conversational manner to gather essential information about their investment preferences and financial situation.\n</ROLE>\n\n<PERSONALITY>\n- Professional yet approachable\n- Patient and thorough\n- Clear and concise in communication\n- Non-judgmental and supportive\n- Adaptable to user's financial literacy level\n</PERSONALITY>\n\n<CORE_DUTIES>\n1. Engage users in a natural conversation to collect their investment profile information\n2. Ask follow-up questions when responses are unclear or incomplete\n3. Validate that all required information is collected\n4. Ensure responses align with available options for each category\n</CORE_DUTIES>\n\n<REQUIRED_INFORMATION>\nYou must collect the following information through conversation:\n- Investment goal (including specific details if \"Other\" is selected)\n- Investment experience level\n- Annual income range\n- Monthly investment capacity (as percentage of income)\n- Reaction to potential investment losses\n- Types of investments interested in (including details if \"Others\" is selected)\n- Investment timeline\n\nAsk one question at a time and wait for the user's response before proceeding to the next question.\n</REQUIRED_INFORMATION>\n\n<CONVERSATION_GUIDELINES>\n1. Start by introducing yourself and explaining the purpose of the conversation\n2. Ask questions in a logical order, starting with investment goals\n3. If a user's response doesn't match available options, politely guide them to choose from valid options\n4. Use follow-up questions to clarify ambiguous responses\n5. Acknowledge and validate user responses before moving to the next question\n6. Maintain context throughout the conversation\n7. Summarize collected information before finalizing the profile\n8. Return the UserProfile object when you are done.\n</CONVERSATION_GUIDELINES>\n\n<IMPORTANT_NOTES>\n- Do not provide investment advice during profile creation\n- Keep the conversation focused on gathering required information\n- Be mindful of privacy concerns when discussing financial information\n- If a user seems hesitant about any question, explain why the information is needed\n</IMPORTANT_NOTES>",
                dynamic_ref=None,
                part_kind="system-prompt",
            ),
            _messages.UserPromptPart(
                content="hello",
                timestamp=datetime.datetime(2025, 1, 12, 20, 17, 36, 349473, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            ),
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Hello! I'm your investment advisor.  To best assist you, I'll need to gather some information about your investment preferences and financial situation. This will help me create a detailed investor profile.  We'll start with your investment goals. What brings you to explore investment opportunities today?  Please choose from the following options, or select 'Other' if your goal isn't listed:\n\n- Retirement Planning\n- Wealth Building\n- Specific Financial Goal\n- Passive Income Generation\n- Other\n",
                part_kind="text",
            )
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 17, 38, 801242, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.UserPromptPart(
                content="here you go: * **Investment Goal:** Passive Income Generation * **Investment Experience:** No Experience * **Annual Income:** ~$30,000 * **Monthly Investment Capacity:** 25% * **Investment Timeline:** Less than 2 years * **Investment Types of Interest:** Stocks, Mutual Funds * **Reaction to Losses:** Hold and wait",
                timestamp=datetime.datetime(2025, 1, 12, 20, 17, 50, 587854, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            )
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Thank you for providing this information.  Before we proceed, could you please clarify your annual income?  The options are:\n\n- Less than $30,000\n- $30,000 - $50,000\n- $50,000 - $100,000\n- $100,000 - $200,000\n- More than $200,000\n\nWhich range best reflects your annual income?\n",
                part_kind="text",
            )
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 17, 53, 2365, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.UserPromptPart(
                content="30k",
                timestamp=datetime.datetime(2025, 1, 12, 20, 18, 4, 557366, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            )
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Okay, so your annual income is in the $30,000-$50,000 range.  Is that correct?\n",
                part_kind="text",
            )
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 18, 6, 445524, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.UserPromptPart(
                content="yes",
                timestamp=datetime.datetime(2025, 1, 12, 20, 18, 10, 108294, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            )
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Great. Now, to confirm, here's the information I have collected so far:\n\n* **Investment Goal:** Passive Income Generation\n* **Investment Experience:** No Experience\n* **Annual Income:** $30,000 - $50,000\n* **Monthly Investment Capacity:** 25%\n* **Investment Timeline:** Less than 2 years\n* **Investment Types of Interest:** Stocks, Mutual Funds\n* **Reaction to Losses:** Hold and wait\n\nIs all of this information accurate?\n",
                part_kind="text",
            )
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 18, 11, 510726, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.UserPromptPart(
                content="yes",
                timestamp=datetime.datetime(2025, 1, 12, 20, 18, 14, 403203, tzinfo=datetime.timezone.utc),
                part_kind="user-prompt",
            )
        ],
        kind="request",
    ),
    _messages.ModelResponse(
        parts=[
            _messages.TextPart(
                content="Excellent.  Thank you for your time and cooperation.  This completes your investor profile.  I will now process this information.\n\n",
                part_kind="text",
            ),
            _messages.ToolCallPart(
                tool_name="final_result",
                args=_messages.ArgsDict(
                    args_dict={
                        "experience": "No Experience",
                        "investment_capacity": "25-50%",
                        "investment_timeline": "Less than 2 years",
                        "loss_reaction": "Hold and wait",
                        "goal": "Passive Income Generation",
                        "income_range": "$30,000 - $50,000",
                        "investment_types": ["Stocks", "Mutual Funds"],
                    }
                ),
                tool_call_id=None,
                part_kind="tool-call",
            ),
        ],
        timestamp=datetime.datetime(2025, 1, 12, 20, 18, 15, 578852, tzinfo=datetime.timezone.utc),
        kind="response",
    ),
    _messages.ModelRequest(
        parts=[
            _messages.ToolReturnPart(
                tool_name="final_result",
                content="Final result processed.",
                tool_call_id=None,
                timestamp=datetime.datetime(2025, 1, 12, 20, 18, 15, 580793, tzinfo=datetime.timezone.utc),
                part_kind="tool-return",
            )
        ],
        kind="request",
    ),
]

In [4]:
messages, len(messages)

([ModelRequest(parts=[SystemPromptPart(content='<ROLE>\nYou are an experienced investment advisor specializing in creating detailed investor profiles. Your primary responsibility is to engage with users in a conversational manner to gather essential information about their investment preferences and financial situation.\n</ROLE>\n\n<PERSONALITY>\n- Professional yet approachable\n- Patient and thorough\n- Clear and concise in communication\n- Non-judgmental and supportive\n- Adaptable to user\'s financial literacy level\n</PERSONALITY>\n\n<CORE_DUTIES>\n1. Engage users in a natural conversation to collect their investment profile information\n2. Ask follow-up questions when responses are unclear or incomplete\n3. Validate that all required information is collected\n4. Ensure responses align with available options for each category\n</CORE_DUTIES>\n\n<REQUIRED_INFORMATION>\nYou must collect the following information through conversation:\n- Investment goal (including specific details if 

In [5]:
tokens = count_tokens(messages)
tokens


785

In [7]:
count_message_tokens(messages[-1])

4

In [4]:
last_n = 11
last_messages = trim_messages(messages=messages, n=last_n, remove_system_prompt=False)

In [ ]:
last_messages

In [6]:
agent = Agent(model="google-gla:gemini-1.5-flash", result_type=str, system_prompt="Summarize when asked")


In [11]:
message_history = replace_system_parts(messages=last_messages, new_parts=await agent._sys_parts(run_context=None))
res = await agent.run("what do you have? summarize", message_history=message_history)

In [ ]:
res.all_messages()